In [ ]:
import osmnx as ox

# Gráfica

In [ ]:
bounds = tuple(float(x) for x in df.to_crs("EPSG:4326").total_bounds)

g = ox.graph_from_bbox(bounds, network_type="drive")
g = ox.project_graph(g, to_crs="EPSG:6372")
g = ox.add_edge_speeds(g)
g = ox.add_edge_travel_times(g)

In [ ]:
coords = df_blocks.centroid.get_coordinates()
df_blocks["node_id"] = ox.nearest_nodes(g, coords["x"], coords["y"])

In [ ]:
df_nodes, df_edges = ox.graph_to_gdfs(g, nodes=True, edges=True)
df_edges = df_edges.reset_index()

In [ ]:
net = pdna.Network(
    df_nodes["x"],
    df_nodes["y"],
    df_edges["u"],
    df_edges["v"],
    df_edges[["travel_time"]],
)
net.precompute(3600)

Generating contraction hierarchies with 1 threads.
Setting CH node vector of size 54312
Setting CH edge vector of size 146950
Range graph removed 130470 edges of 293900
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


# Accesibilidad

In [ ]:
def calculate_weighted_accessibility(
    net: pdna.Network,
    coords: pd.DataFrame,
    poi_type: str,
    df_blocks: gpd.GeoDataFrame,
    *,
    weight_col: str = "POBTOT",
) -> pd.Series:
    net.set_pois(poi_type, 3600, 1, coords["x"], coords["y"])
    res = (
        net.nearest_pois(3600, poi_type)
        .rename(columns={1: "travel_time"})
        .reset_index(names="node_id")
    )
    return (
        df_blocks.merge(res, on="node_id", how="left")
        .assign(num=lambda df: df[weight_col] * df["travel_time"])
        .groupby("CVEGEO_AGEB")
        .agg(
            {
                "num": "sum",
                weight_col: "sum",
            },
        )
        .assign(**{f"{poi_type}_travel_time": lambda df: df["num"] / df[weight_col]})[
            f"{poi_type}_travel_time"
        ]
        .rename_axis("CVEGEO")
    )

## Hospitales

In [ ]:
rows["tiempo_viaje_hospitales"] = (
    calculate_weighted_accessibility(
        net,
        df_hospitals.get_coordinates(),
        "hospital",
        df_blocks,
    )
    / 60
)

## Preparatorias

In [ ]:
rows["tiempo_viaje_preparatorias"] = (
    calculate_weighted_accessibility(
        net,
        df_highschools.get_coordinates(),
        "highschool",
        df_blocks,
    )
    / 60
)